In [11]:
# Pre-flight checks (quotas, permissions, users, etc.)
# https://docs.openshift.com/rosa/rosa_cli/rosa-get-started-cli.html#rosa-init_rosa-getting-started-cli
!rosa init -y

I: Logged in as 'faermanj' on 'https://api.openshift.com'
I: Validating AWS credentials...
I: AWS credentials are valid!
I: Verifying permissions for non-STS clusters
I: Validating SCP policies...
I: AWS SCP policies ok
I: Validating AWS quota...
I: AWS quota ok. If cluster installation fails, validate actual AWS resource usage against https://docs.openshift.com/rosa/rosa_getting_started/rosa-required-aws-service-quotas.html
I: Ensuring cluster administrator user 'osdCcsAdmin'...
I: Admin user 'osdCcsAdmin' created successfully!
I: Validating SCP policies for 'osdCcsAdmin'...
I: AWS SCP policies ok
I: Validating cluster creation...
I: Cluster creation valid
I: Verifying whether OpenShift command-line tool is available...
I: Current OpenShift Client Version: 4.8.9


In [8]:
# Use a unique id to identify resources)
import os
unique_id = os.getenv("UNIQUE_ID","rhnb")
# Define cluster name
cluster_name=unique_id+"-rosa"
cluster_name

'rhnb-rosa'

In [19]:
# Before provisioning a cluster, make sure to understand pricing and security implications.
# https://aws.amazon.com/rosa/pricing/

In [12]:
# Create a cluster with default settings
! rosa create cluster --cluster-name={cluster_name}

I: Creating cluster 'rhnb-rosa'
I: To view a list of clusters and their status, run 'rosa list clusters'
I: Cluster 'rhnb-rosa' has been created.
I: Once the cluster is installed you will need to add an Identity Provider before you can login into the cluster. See 'rosa create idp --help' for more information.
I: To determine when your cluster is Ready, run 'rosa describe cluster -c rhnb-rosa'.
I: To watch your cluster installation logs, run 'rosa logs install -c rhnb-rosa --watch'.
Name:                       rhnb-rosa
ID:                         1q7v8mnfd4q82qnu7fn42ts360bm88l9
External ID:                
OpenShift Version:          
Channel Group:              stable
DNS:                        rhnb-rosa.awcz.p1.openshiftapps.com
AWS Account:                269733383066
API URL:                    
Console URL:                
Region:                     ap-northeast-1
Multi-AZ:                   false
Nodes:
 - Control plane:           3
 - Infra:                   2
 - Compute:   

In [8]:
! rosa list clusters --output json | jq -r '.[0]'

{
  "kind": "Cluster",
  "id": "1pqsi4h6ufunjecjvc1e1a89d57i55uv",
  "href": "/api/clusters_mgmt/v1/clusters/1pqsi4h6ufunjecjvc1e1a89d57i55uv",
  "api": {
    "url": "https://api.rhnb-rosa.iyei.p1.openshiftapps.com:6443",
    "listening": "external"
  },
  "aws": {
    "private_link": false
  },
  "aws_infrastructure_access_role_grants": {
    "items": []
  },
  "ccs": {
    "kind": "CCS",
    "disable_scp_checks": false,
    "enabled": true
  },
  "dns": {
    "base_domain": "iyei.p1.openshiftapps.com"
  },
  "addons": {
    "items": []
  },
  "billing_model": "standard",
  "cloud_provider": {
    "kind": "CloudProviderLink",
    "id": "aws",
    "href": "/api/clusters_mgmt/v1/cloud_providers/aws"
  },
  "console": {
    "url": "https://console-openshift-console.apps.rhnb-rosa.iyei.p1.openshiftapps.com"
  },
  "creation_timestamp": "2022-01-19T13:50:15Z",
  "disable_user_workload_monitoring": false,
  "display_name": "rhnb-rosa",
  "etcd_encryption": true,
  "external_id": "d1e3dfd8-8

In [13]:
# Wait until cluster is ready
# TODO: Use waiter from https://github.com/openshift/rosa/pull/503
# rosa wait-for -c id42-cluster
states=! rosa list clusters --output json | jq -r ".[].status.state"
states=states.s
states

'installing'

Installing a cluster with default settings takes around half an hour.

Good time to watch a video on the [OpenShift Youtube channel](https://www.youtube.com/c/OpenShift) :)

In [5]:
# Create admin user
# TODO: Use password option from PR (-p Masterkey123)
! rosa create admin --cluster="{cluster_name}" | tee create_admin.log

W: It is recommended to add an identity provider to login to this cluster. See 'rosa create idp --help' for more information.
I: Admin account has been added to cluster 'rhnb-rosax'.
I: Please securely store this generated password. If you lose this password you can delete and recreate the cluster admin user.
I: To login, run the following command:

   oc login https://api.rhnb-rosax.ieu5.p1.openshiftapps.com:6443 --username cluster-admin --password cgpft-gWAkQ-AzurJ-4gno9

I: It may take up to a minute for the account to become active.


In [6]:
# Check login command
# TODO: Use choice of password
!cat create_admin.log 

I: Admin account has been added to cluster 'rhnb-rosax'.
I: Please securely store this generated password. If you lose this password you can delete and recreate the cluster admin user.
I: To login, run the following command:

   oc login https://api.rhnb-rosax.ieu5.p1.openshiftapps.com:6443 --username cluster-admin --password cgpft-gWAkQ-AzurJ-4gno9

I: It may take up to a minute for the account to become active.


In [ ]:
# Wait for admin user to be active, usually takes a couple minutes
# TODO: Add a waiter command for that

In [ ]:
# Login (--insecure-skip-tls-verify=true)
! oc login https://api.rhnb-rosa.b1w6.p1.openshiftapps.com:6443 --username cluster-admin --password B46vg-UEfUS-hmQjc-8SFbs --insecure-skip-tls-verify=true

In [ ]:
# Verify cluster authentication
! oc whoami

In [ ]:
# Ping for version
!oc version

In [ ]:
# Check cluster status
!oc status

In [ ]:
# Check running resources
!oc get nodes

In [ ]:
# Decomissioning

In [7]:
# delete cluster
!rosa delete cluster -y -c "{cluster_name}"

I: Cluster 'rhnb-rosax' will start uninstalling now
I: To watch your cluster uninstallation logs, run 'rosa logs uninstall -c rhnb-rosax --watch'


In [ ]:
#  delete IAM admin user
!rosa init -y --delete